In [2]:
import pandas as pd
import numpy as np
import os
import json
from pathlib import Path

# Set up paths - using absolute paths for reliability
raw_data_path = os.path.abspath("/root/nfs/AJ FinRag/Test Data/all_companies.csv")
processed_data_dir = os.path.abspath("/root/nfs/AJ FinRag/Test Data/Company Processed Test Data")

def verify_file_exists(file_path):
    """Check if file exists and is accessible"""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Input file not found at: {file_path}")
    if not os.path.isfile(file_path):
        raise ValueError(f"Path exists but is not a file: {file_path}")
    if not os.access(file_path, os.R_OK):
        raise PermissionError(f"Cannot read file: {file_path}")

def clean_and_drop_nan(df):
    """Clean and drop NaN values while preserving data structure"""
    # First replace inf/-inf with NaN
    df = df.replace([np.inf, -np.inf], np.nan)

    # For indicator columns, we can safely drop rows where all indicator values are NaN
    indicator_cols = ['MACD_Histogram', 'macd_crossover', 'bollinger_bands',
                     'exceeding_upper', 'exceeding_lower',
                     'overbought_and_oversold_conditions', 'kdj_crossover',
                     'Returns', 'VWAP', 'alpha_smr', 'alpha_mom']

    # For price/volume data, we should keep even if indicators are NaN
    price_cols = ['open', 'high', 'low', 'close', 'volume', 'adj_close']

    # Keep rows where we have at least one indicator value or all price data
    df = df.dropna(subset=price_cols, how='all')

    # For indicators, only drop rows where all indicators are NaN
    existing_indicator_cols = [col for col in indicator_cols if col in df.columns]
    if len(existing_indicator_cols) > 0:
        df = df.dropna(subset=existing_indicator_cols, how='all')

    return df

def clean_nan_values(df):
    """Replace all NaN/NaT values with None for better JSON serialization"""
    return df.where(pd.notnull(df), None)

def calculate_ema(prices, span):
    """Calculate Exponential Moving Average"""
    return prices.ewm(span=span, adjust=False).mean()

def calculate_macd(df, slow=26, fast=12, signal=9):
    """Calculate MACD indicator"""
    df = df.copy()
    
    # Calculate EMAs
    ema_fast = calculate_ema(df['adj_close'], fast)
    ema_slow = calculate_ema(df['adj_close'], slow)
    
    # MACD line
    macd_line = ema_fast - ema_slow
    
    # Signal line
    signal_line = calculate_ema(macd_line, signal)
    
    # MACD Histogram
    df['MACD_Histogram'] = macd_line - signal_line
    
    # MACD crossover signals
    ema_signal = np.where(ema_fast > ema_slow, 1, -1)
    ema_signal_change = pd.Series(ema_signal).diff()
    
    conditions = [ema_signal_change == 2, ema_signal_change == -2]
    choices = ['bullish_cross', 'bearish_cross']
    df['macd_crossover'] = np.select(conditions, choices, default=None)
    
    return df

def calculate_bollinger_bands(df, window=20):
    """Calculate Bollinger Bands"""
    df = df.copy()
    
    # Calculate moving average and standard deviation
    middle_band = df['adj_close'].rolling(window=window).mean()
    std_dev = df['adj_close'].rolling(window=window).std()
    upper_band = middle_band + (std_dev * 2)
    lower_band = middle_band - (std_dev * 2)
    
    # Bollinger Band position
    conditions = [df['adj_close'] > upper_band, df['adj_close'] < lower_band]
    choices = ['exceeding_upper', 'exceeding_lower']
    df['bollinger_bands'] = np.select(conditions, choices, default=None)
    
    # Exceeding values
    df['exceeding_upper'] = np.select([df['adj_close'] > upper_band], 
                                     [df['adj_close'] - upper_band], default=None)
    df['exceeding_lower'] = np.select([df['adj_close'] < lower_band], 
                                     [df['adj_close'] - lower_band], default=None)
    
    return df

def calculate_kdj(df, n=9):
    """Calculate KDJ indicator"""
    df = df.copy()
    
    # Calculate RSV (Raw Stochastic Value)
    low_n = df['low'].rolling(window=n).min()
    high_n = df['high'].rolling(window=n).max()
    rsv = (df['close'] - low_n) / (high_n - low_n) * 100
    
    # Calculate K, D, J
    k = rsv.ewm(alpha=1/3, adjust=False).mean()
    d = k.ewm(alpha=1/3, adjust=False).mean()
    j = 3 * k - 2 * d
    
    # Overbought/Oversold conditions
    conditions1 = [(k > 80) & (d > 70) & (j > 90), (k < 20) & (d < 30)]
    choices1 = ['overbought_area', 'oversold_area']
    df['overbought_and_oversold_conditions'] = np.select(conditions1, choices1, default=None)
    
    # KDJ crossover signals
    conditions2 = [k > d, k < d]
    choices2 = ['bullish_signal', 'bearish_signal']
    df['kdj_crossover'] = np.select(conditions2, choices2, default=None)
    
    return df

def calculate_returns(df):
    """Calculate daily returns"""
    df = df.copy()
    df['Returns'] = df['adj_close'].pct_change()
    return df

def calculate_vwap(df):
    """Calculate VWAP (Volume Weighted Average Price)"""
    df = df.copy()
    
    # Typical price
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    
    # VWAP calculation - using cumulative approach
    df['VWAP'] = (typical_price * df['volume']).cumsum() / df['volume'].cumsum()
    
    # Handle division by zero
    df['VWAP'] = df['VWAP'].fillna(typical_price)
    
    return df

def add_mean_reversion_alpha(df):
    """Add mean reversion alpha factor"""
    df = df.copy()
    
    # Mean reversion factor: -log(open / previous_close)
    df['alpha_smr'] = -np.log(df['open'] / df['close'].shift(1))
    
    # Handle infinite values
    df['alpha_smr'] = df['alpha_smr'].replace([np.inf, -np.inf], np.nan)
    
    return df

def add_momentum_alpha(df):
    """Add momentum alpha factor"""
    df = df.copy()
    
    # Momentum factor: log(previous_close / previous_open)
    df['alpha_mom'] = np.log(df['close'].shift(1) / df['open'].shift(1))
    
    # Handle infinite values
    df['alpha_mom'] = df['alpha_mom'].replace([np.inf, -np.inf], np.nan)
    
    return df

def calculate_all_indicators(df):
    """Calculate all technical indicators for a dataframe"""
    # Make sure data is sorted by date for each ticker
    df = df.sort_values(['ticker', 'date']).reset_index(drop=True)
    df = calculate_macd(df)
    df = calculate_bollinger_bands(df)
    df = calculate_kdj(df)
    df = calculate_returns(df)
    df = calculate_vwap(df)
    df = add_mean_reversion_alpha(df)
    df = add_momentum_alpha(df)
    df = clean_and_drop_nan(df)
    df = clean_nan_values(df)  # Additional cleaning for JSON serialization
    
    return df

def process_dataframe_with_indicators(input_path, output_dir):
    """Process existing dataframe by adding technical indicators"""
    try:
        # Verify input file exists
        verify_file_exists(input_path)

        # Create output directory if it doesn't exist
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        # Load the combined data
        print(f"Loading data from: {input_path}")
        df = pd.read_csv(input_path)

        # Convert date column to datetime if it's not already
        if not pd.api.types.is_datetime64_any_dtype(df['date']):
            df['date'] = pd.to_datetime(df['date'])

        # Process each company separately
        processed_dfs = []
        tickers = df['ticker'].unique()

        print(f"Processing {len(tickers)} companies...")

        for ticker in tickers:
            print(f"\\nProcessing indicators for {ticker}...")
            try:
                # Get data for this ticker
                ticker_data = df[df['ticker'] == ticker].copy()

                # Sort by date
                ticker_data = ticker_data.sort_values('date').reset_index(drop=True)
                
                print(f"  Data shape: {ticker_data.shape}")
                print(f"  Date range: {ticker_data['date'].min()} to {ticker_data['date'].max()}")

                # Calculate indicators
                processed_df = calculate_all_indicators(ticker_data)

                # Save individual company file (CSV)
                output_csv_path = os.path.join(output_dir, f"{ticker}_processed.csv")
                processed_df.to_csv(output_csv_path, index=False)
                print(f"  Saved CSV: {output_csv_path}")

                # Save individual company file (JSON)
                output_json_path = os.path.join(output_dir, f"{ticker}_processed.json")
                processed_df.to_json(output_json_path, orient='records', lines=True)
                print(f"  Saved JSON: {output_json_path}")

                processed_dfs.append(processed_df)
                print(f"  {ticker} completed successfully")
                
            except Exception as e:
                print(f" Error processing {ticker}: {str(e)}")
                import traceback
                traceback.print_exc()
                continue

        # Combine all processed data
        if processed_dfs:
            print(f"\\nCombining data from {len(processed_dfs)} companies...")
            combined_df = pd.concat(processed_dfs, ignore_index=True)
            combined_df = clean_nan_values(combined_df)  # Final cleaning

            # Save combined data (CSV)
            combined_csv_path = os.path.join(output_dir, "all_companies_processed.csv")
            combined_df.to_csv(combined_csv_path, index=False)
            print(f" Successfully saved combined CSV to {combined_csv_path}")

            # Save combined data (JSON)
            combined_json_path = os.path.join(output_dir, "all_companies_processed.json")
            combined_df.to_json(combined_json_path, orient='records', lines=True)
            print(f" Successfully saved combined JSON to {combined_json_path}")

            return combined_df
        else:
            print("\\n No data was processed successfully")
            return None

    except Exception as e:
        print(f"\\n Fatal error in processing: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

if __name__ == "__main__":
    print("=== Starting Technical Indicator Processing ===")
    print(f"Input file: {raw_data_path}")
    print(f"Output directory: {processed_data_dir}")

    try:
        processed_df = process_dataframe_with_indicators(raw_data_path, processed_data_dir)

        if processed_df is not None:
            print("\\n Processing complete! Sample of processed data:")
            print(f"Shape: {processed_df.shape}")
            print("\\nColumns:", processed_df.columns.tolist())
            print("\\nSample data:")
            print(processed_df.head())

            print("\\nData summary:")
            print(f"Date range: {processed_df['date'].min()} to {processed_df['date'].max()}")
            print(f"Companies: {processed_df['ticker'].nunique()}")
            print(f"Total records: {len(processed_df)}")

            # Show NaN statistics
            print("\\nNaN values in each column:")
            nan_counts = processed_df.isna().sum()
            for col, count in nan_counts.items():
                if count > 0:
                    print(f"  {col}: {count}")
                    
            # Show indicator statistics
            print("\\nIndicator value counts:")
            indicator_cols = ['MACD_Histogram', 'macd_crossover', 'bollinger_bands',
                             'exceeding_upper', 'exceeding_lower',
                             'overbought_and_oversold_conditions', 'kdj_crossover',
                             'Returns', 'VWAP', 'alpha_smr', 'alpha_mom']
            
            for col in indicator_cols:
                if col in processed_df.columns:
                    non_null_count = processed_df[col].notna().sum()
                    print(f"  {col}: {non_null_count} non-null values")
        else:
            print("\\n Processing completed with errors")
    except Exception as e:
        print(f"\\n Failed to run processing: {str(e)}")
        import traceback
        traceback.print_exc()

=== Starting Technical Indicator Processing ===
Input file: /root/nfs/AJ FinRag/Test Data/all_companies.csv
Output directory: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data
Loading data from: /root/nfs/AJ FinRag/Test Data/all_companies.csv
Processing 50 companies...
\nProcessing indicators for AAPL...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/AAPL_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/AAPL_processed.json
  AAPL completed successfully
\nProcessing indicators for MSFT...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/MSFT_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/MSFT_processed.json
  MSFT completed successfully
\nProcessing indicators for GOOGL...
  Data shape: (80, 8)
  Date ra

/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.inf, -np.inf], np.nan)
/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.inf, -np.inf], np.nan)
/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no

  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/AMZN_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/AMZN_processed.json
  AMZN completed successfully
\nProcessing indicators for META...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/META_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/META_processed.json
  META completed successfully
\nProcessing indicators for TSLA...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/TSLA_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/TSLA_processed.json
  TSLA completed successfully
\nProcessing indicators for NVDA...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/

/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.inf, -np.inf], np.nan)
/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.inf, -np.inf], np.nan)
/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no

  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/ADBE_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/ADBE_processed.json
  ADBE completed successfully
\nProcessing indicators for CRM...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/CRM_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/CRM_processed.json
  CRM completed successfully
\nProcessing indicators for ORCL...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/ORCL_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/ORCL_processed.json
  ORCL completed successfully
\nProcessing indicators for IBM...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/A

...

  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/MRK_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/MRK_processed.json
  MRK completed successfully
\nProcessing indicators for ABT...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/ABT_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/ABT_processed.json
  ABT completed successfully
\nProcessing indicators for UNH...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/UNH_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/UNH_processed.json
  UNH completed successfully
\nProcessing indicators for LLY...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRa

/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.inf, -np.inf], np.nan)
/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.inf, -np.inf], np.nan)
/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no

  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/PG_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/PG_processed.json
  PG completed successfully
\nProcessing indicators for KO...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/KO_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/KO_processed.json
  KO completed successfully
\nProcessing indicators for PEP...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/PEP_processed.csv
  Saved JSON: /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/PEP_processed.json
  PEP completed successfully
\nProcessing indicators for XOM...
  Data shape: (80, 8)
  Date range: 2025-01-02 00:00:00 to 2025-04-29 00:00:00
  Saved CSV: /root/nfs/AJ FinRag/Test 

/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.inf, -np.inf], np.nan)
/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.inf, -np.inf], np.nan)
/tmp/ipykernel_128/1104425001.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no

 Successfully saved combined CSV to /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/all_companies_processed.csv
 Successfully saved combined JSON to /root/nfs/AJ FinRag/Test Data/Company Processed Test Data/all_companies_processed.json
\n Processing complete! Sample of processed data:
Shape: (4000, 19)
\nColumns: ['date', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'adj_close', 'MACD_Histogram', 'macd_crossover', 'bollinger_bands', 'exceeding_upper', 'exceeding_lower', 'overbought_and_oversold_conditions', 'kdj_crossover', 'Returns', 'VWAP', 'alpha_smr', 'alpha_mom']
\nSample data:
        date ticker        open        high         low       close    volume  \
0 2025-01-02   AAPL  248.929993  249.100006  241.820007  243.850006  55740700   
1 2025-01-03   AAPL  243.360001  244.179993  241.889999  243.360001  40244100   
2 2025-01-06   AAPL  244.309998  247.330002  243.199997  245.000000  45045600   
3 2025-01-07   AAPL  242.979996  245.550003  241.350006  242.210007  